# Remove all geometry and layers below L2

## Imports

In [1]:
import os
from ansys.edb.database import Database
from ansys.edb.layer import StackupLayer, LayerType, LayerCollectionMode, LayerCollection
from ansys.edb.layout import Cell, CellType
from ansys.edb.net import Net
from ansys.edb.primitive import Rectangle, RectangleRepresentationType, Circle, Text
from ansys.edb.session import session, launch_session

## Configs

In [2]:
# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = None
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "remove_all_geometry_and_layers_below_L2.aedb")

## Launch a session

In [3]:
# Make sure old one has been disconnected prior to starting a new one, if any
session = launch_session(RPC_SERVER, 50051)

## Basic setup of empty EDB

In [4]:
"""Create Database, Cell, Layout, Net"""
db = Database.create(EDB_FILE)
cell = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign1")
layout = cell.layout
net = Net.create(layout, 'GND')

id: 1



## Setup Layers for cell

In [5]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layer collection
layer_gnd = StackupLayer.create('Ground', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
insulator_layer_1 = StackupLayer.create('S1', LayerType.DIELECTRIC_LAYER, 0.0002, 0.0002, "FR4-Epoxy")
layer_1 = StackupLayer.create('L1', LayerType.SIGNAL_LAYER, 0.0001, 0.0003, "copper")
insulator_layer_2 = StackupLayer.create('S2', LayerType.DIELECTRIC_LAYER, 0.0001, 0.0004, "FR4-Epoxy")
layer_2 = StackupLayer.create('L2', LayerType.SIGNAL_LAYER, 0.0001, 0.0005, "copper")
insulator_layer_3 = StackupLayer.create('S3', LayerType.DIELECTRIC_LAYER, 0.0001, 0.0005, "FR4-Epoxy")
layer_3 = StackupLayer.create('L3', LayerType.SIGNAL_LAYER, 0.0001, 0.0007, "copper")

# Layer Collection Setup
lc = layout.layer_collection
lc.mode = LayerCollectionMode.OVERLAPPING
lc.add_layers([layer_gnd, insulator_layer_1, layer_1, insulator_layer_2, layer_2, insulator_layer_3, layer_3])

# Retrieve layer collection and layers
layout.layer_collection = lc
insulator_layer_1 = lc.find_by_name('S1')
layer_1 = lc.find_by_name('L1')
insulator_layer_2 = lc.find_by_name('S2')
layer_2 = lc.find_by_name('L2')
insulator_layer_3 = lc.find_by_name('S3')
layer_3 = lc.find_by_name('L3')
lc = layout.layer_collection

# Priint layers created
print([(l.name) for l in lc.get_layers() ])

['L3', 'S3', 'L2', 'S2', 'L1', 'S1', 'Ground']


## Create geometry

In [6]:
"""Create Primitive objects in different layers"""

# Multiply each value with a resolution
um = lambda val : val * 1e-6

rect = Rectangle.create(layout=layout,
                        layer=insulator_layer_1,
                        net=net,
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(10),
                        param2=um(10),
                        param3=um(20),
                        param4=um(20),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout,
                          layer=layer_1,
                          net=net,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(0.0),
                          param2=um(0.0),
                          param3=um(30),
                          param4=um(30),
                          corner_rad=0.0,
                          rotation=0.0
                          )

text = Text.create(layout=layout, layer=insulator_layer_2, center_x=um(0.0), center_y=um(0.0), text="my Text")

rect_3 = Rectangle.create(layout=layout,
                          layer=layer_2,
                          net=net,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(15),
                          param2=um(15),
                          param3=um(25),
                          param4=um(25),
                          corner_rad=0.0,
                          rotation=0.0
                          )

text = Text.create(layout=layout, layer=insulator_layer_3, center_x=um(3.0), center_y=um(3.0), text="my Text 2")

circle = Circle.create(layout=layout, layer=layer_3, net=net, center_x=um(0.0), center_y=um(0.0), radius=um(34.0))

#Retrieve the primitives
primitives_created = layout.primitives

# Print primitives created
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('RECTANGLE', 'S1'), ('RECTANGLE', 'L1'), ('TEXT', 'S2'), ('RECTANGLE', 'L2'), ('TEXT', 'S3'), ('CIRCLE', 'L3')]


## Identify layers above L2

In [7]:
"""Create a list of layers to be kept"""

l2 = layout.layer_collection.find_by_name('L2')
layers_to_keep = []
for layer in lc.get_layers():
    if not layer.lower_elevation < l2.lower_elevation:
        layers_to_keep.append(layer)
        
# Print layers
print([(l.name) for l in layers_to_keep])

['L3', 'S3', 'L2']


## Delete geometry of Layers which will be deleted

In [8]:
"""Delete geometry of Layers under L2"""
for primitive in primitives_created:
    if not(primitive.layer.id in [l.id for l in layers_to_keep]):
        primitive.delete()

#Retrieve the primitives again
primitives_kept = layout.primitives

# Print primitives kept
print([(p.primitive_type.name, p.layer.name) for p in primitives_kept])

[('RECTANGLE', 'L2'), ('TEXT', 'S3'), ('CIRCLE', 'L3')]


## Create and Set the new layer collection

In [9]:
"""Assign the new layer collection to the layout"""

new_lc = LayerCollection.create(layout.layer_collection.mode)
new_lc.add_layers(layers_to_keep)
layout.layer_collection = new_lc

# Save database

In [10]:
"""Save database if you wish."""

db.save()

# Disconnect from database

In [11]:
"""Don't forget to disconnect it!"""

session.disconnect()